In [1]:
from google.colab import files, drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 4.6MB/s 
     |████████████████████████████████| 3.2MB 18.8MB/s 
     |████████████████████████████████| 890kB 38.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=3c8f6e9bd31ef3ab395f27d87161f1cb5afe9b333909f042753310cf823512f5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

In [4]:
import pickle
import os
def save_obj(obj,path,name):
    with open(os.path.join(path, name + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [5]:
data_text_features = load_obj("/content/drive/MyDrive/DDI/training/data_text_features.pkl")

In [6]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [8]:
def mean_pooling_2(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, [0,1])
    sum_mask = torch.clamp(input_mask_expanded.sum([0,1]), min=1e-9)
    return sum_embeddings.unsqueeze(0) / sum_mask.unsqueeze(0)

In [9]:
#Sentences we want sentence embeddings for
sentences = ['This framework generates embeddings for each input sentence',
             'Sentences are passed as a list of string.',
             'The quick brown fox jumps over the lazy dog when it is raining.']

In [10]:

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")

model = AutoModel.from_pretrained("dmis-lab/biobert-v1.1")

In [11]:
#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=256, return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [ ]:
for sl in range(220,len(data_text_features),100):
  bert_embedding=[]
  bert_embedding_2 = []
  for i in range(sl,sl+100):
      data_list = []
      data_list_2 = []
      for j in range(2,len(data_text_features[i])):
          if j == 4:
  #             print(i)
  #             print(j)
              striped_list = [_.strip() for _ in data_text_features[i][j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              
              if len(full_sent.split()) > 512:
                  encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1].mean(0,keepdim=True))
              else:
                  encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1])
              continue
          elif j ==5:
              striped_list = [_.strip() for _ in data_text_features[i][j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              if len(full_sent.split()) > 512:
                  encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1].mean(0,keepdim=True))
              else:
                  encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1])
              continue
          elif j == 6:
              striped_list = [_.strip() for _ in data_text_features[i][j]]
              if len(striped_list) == 0: continue
              encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
              #Compute token embeddings
              with torch.no_grad():
                  model_output = model(**encoded_input)
              #Perform pooling. In this case, mean pooling
              sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
              data_list.append(sentence_embeddings)
              data_list_2.append(model_output[1].mean(0,keepdim=True))
              continue
          else:
              striped_list = [_.strip() for _ in data_text_features[i][j]]
              if len(striped_list) == 0: continue
              encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
              #Compute token embeddings
              with torch.no_grad():
                  model_output = model(**encoded_input)
              #Perform pooling. In this case, mean pooling
              sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
              data_list.append(sentence_embeddings)
              data_list_2.append(model_output[1])
              continue
      bert_embedding.append(np.array(torch.cat(data_list,0)))
      bert_embedding_2.append(np.array(torch.cat(data_list_2,0)))
  bert_np = np.array(bert_embedding)
  bert_np_2 = np.array(bert_embedding_2)
  savedex = (sl-220)//100 +1
  np.save("/content/drive/MyDrive/DDI/training/BioBert/"+str(savedex),bert_np)
  np.save("/content/drive/MyDrive/DDI/training/BioBert_2/"+str(savedex),bert_np_2)

In [18]:
bert_np = np.array(bert_embedding)
bert_np_2 = np.array(bert_embedding_2)
np.save("/content/drive/MyDrive/DDI/training/BioBert/0",bert_np[:-1,:,:])
np.save("/content/drive/MyDrive/DDI/training/BioBert_2/0",bert_np_2[:-1,:,:])

In [19]:
bert_np[:-1,:,:].shape

(220, 5, 768)